<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/277_MissionOrchestratorAgent_Toolshed_ReportGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Report generation utilities

In [ ]:
"""Report generation utilities"""

from typing import Dict, Any
from datetime import datetime


def generate_mission_report(state: Dict[str, Any]) -> str:
    """
    Generate a comprehensive markdown report for the mission.

    This is a generic implementation that works with any orchestrator state
    containing standard fields: mission, executed_tasks, kpi_metrics, etc.

    Args:
        state: Complete orchestrator state

    Returns:
        Markdown report string
    """
    mission = state.get("mission", {})
    executed_tasks = state.get("executed_tasks", [])
    mission_kpis = state.get("mission_kpis", {})
    kpi_metrics = state.get("kpi_metrics", {})
    kpi_status = state.get("kpi_status", {})
    approval_history = state.get("approval_history", [])
    pending_approvals = state.get("pending_approvals", [])

    report = f"""# Mission Execution Report

**Mission:** {mission.get('mission_name', 'Unknown')}
**Mission ID:** {mission.get('mission_id', 'Unknown')}
**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

---

## Executive Summary

**Status:** {state.get('mission_status', 'unknown')}
**Progress:** {state.get('progress_percentage', 0):.1f}%
**Tasks Completed:** {state.get('tasks_completed', 0)}/{state.get('tasks_total', 0)}
**Elapsed Time:** {state.get('elapsed_time_minutes', 0):.2f} minutes

---

## Mission Overview

**Description:** {mission.get('description', 'N/A')}

**Objective:** {state.get('goal', {}).get('objective', 'N/A')}

---

## Task Execution Summary

"""

    # Task execution details
    if executed_tasks:
        report += "### Completed Tasks\n\n"
        for i, task in enumerate(executed_tasks, 1):
            task_id = task.get('task_id', 'Unknown')
            task_name = task.get('task', 'Unknown')
            agent_name = task.get('agent_name', 'Unknown')
            status = task.get('status', 'unknown')
            duration = task.get('duration_minutes', 0)
            requires_approval = task.get('requires_approval', False)

            approval_status = ""
            if requires_approval:
                is_approved = any(
                    a.get('task_id') == task_id and a.get('decision') == 'approved'
                    for a in approval_history
                )
                approval_status = " ✓ Approved" if is_approved else " ⚠ Pending Approval"

            report += f"{i}. **{task_id}**: {task_name}\n"
            report += f"   - Agent: {agent_name}\n"
            report += f"   - Status: {status}{approval_status}\n"
            report += f"   - Duration: {duration:.2f} minutes\n\n"
    else:
        report += "No tasks executed.\n\n"

    # Pending approvals
    if pending_approvals:
        report += "### Pending Approvals\n\n"
        for req in pending_approvals:
            report += f"- **{req.get('task_id')}**: {req.get('task', 'Unknown')}\n"
            report += f"  - Requested: {req.get('requested_at', 'N/A')}\n\n"

    # KPI Metrics
    report += "---\n\n## KPI Metrics\n\n"

    if kpi_metrics:
        report += "### Current Performance\n\n"
        for key, value in kpi_metrics.items():
            if key != "improvement_percentage":
                report += f"- **{key.replace('_', ' ').title()}**: {value}\n"

        if "improvement_percentage" in kpi_metrics:
            improvement = kpi_metrics["improvement_percentage"]
            report += f"\n**Improvement vs Baseline:** {improvement:.1f}%\n"

    # KPI Status
    if kpi_status:
        report += "\n### KPI Status\n\n"
        for key, status in kpi_status.items():
            status_icon = "✓" if status in ["on_track", "exceeded"] else "⚠"
            report += f"{status_icon} **{key.replace('_', ' ').title()}**: {status}\n"

    # Baseline comparison
    if mission_kpis:
        report += "\n### Target vs Baseline\n\n"
        target_keys = [k for k in mission_kpis.keys() if k.startswith('target_')]

        for target_key in target_keys:
            target = mission_kpis.get(target_key)
            baseline_key = target_key.replace('target_', 'baseline_')
            baseline = mission_kpis.get(baseline_key)

            if target is not None:
                report += f"- **{target_key.replace('_', ' ').title()}**: {target}"
                if baseline is not None:
                    report += f" (Baseline: {baseline})"
                report += "\n"

    # Approval History
    if approval_history:
        report += "\n---\n\n## Approval History\n\n"
        for approval in approval_history:
            decision_icon = "✓" if approval.get('decision') == 'approved' else "✗"
            report += f"{decision_icon} **{approval.get('task_id')}**: {approval.get('decision', 'unknown')}"
            report += f" by {approval.get('decided_by', 'unknown')}"
            report += f" at {approval.get('decided_at', 'N/A')}\n"

    # Errors
    errors = state.get('errors', [])
    if errors:
        report += "\n---\n\n## Errors\n\n"
        for error in errors:
            report += f"- ⚠ {error}\n"

    # Footer
    report += f"\n---\n\n*Report generated by Orchestrator Agent*\n"

    return report





# ✅ Report Generation Utility — Summary

### **`generate_mission_report()`**

This utility creates a full **Markdown report** that explains everything the orchestrator did during the mission. It pulls information from the agent’s state and organizes it into a polished, human-readable document.

Think of it as the agent writing its own **after-action report**.

---

# ⭐ What It Includes

The function builds a report with several important sections:

### **1. Mission Metadata**

* Mission name
* Mission ID
* Timestamp
* Description
* Objective

This orients the reader.

---

### **2. Executive Summary**

Shows quick, high-level indicators:

* Mission status
* Progress %
* Tasks completed out of total
* Elapsed time

This is the “at a glance” dashboard for stakeholders.

---

### **3. Task Execution Summary**

Lists every executed task:

* Task ID and name
* Which agent performed it
* Status (completed, failed, pending approval)
* Duration
* Approval indicator (✓ Approved or ⚠ Pending)

This gives a traceable log of what happened.

---

### **4. Pending Approvals**

If any tasks still require HITL, they show up here.

This instantly tells humans where attention is needed.

---

### **5. KPI Metrics & KPI Status**

Two layers:

* **Metrics:** actual numbers (time, steps, pipeline days, etc.)
* **Status:** whether each KPI is *on_track*, *exceeded*, or *at_risk*

This ties the mission’s execution to measurable business value.

---

### **6. Baseline vs Target**

Shows:

* Each KPI target
* The original baseline (if available)

This helps communicate the improvement achieved.

---

### **7. Approval History**

A chronological log of approvals:

* Task ID
* Decision (approved/rejected)
* Who approved
* When they approved

This is essential for audit trails and trust.

---

### **8. Errors**

If anything went wrong in any node, the report lists it clearly.

This is extremely helpful for debugging and transparency.

---

### **9. Footer**

Just a simple signature line.

---

# ⭐ Why This Utility Is Important

This function turns the orchestrator’s internal operations into a **professional, explainable artifact**.

### It provides:

* Full transparency
* Easy debugging
* Stakeholder communication
* Compliance-friendly auditability
* A consistent reporting format across all missions

It also slots perfectly into the toolshed because **any agent workflow benefits from report generation**, not just mission orchestrators.

---

# 🎯 In one sentence

**`generate_mission_report()` turns the agent’s entire mission into a clear, comprehensive story that humans can understand and trust.**




In [ ]:
"""Report file handling utilities"""

from pathlib import Path
from datetime import datetime


def save_report(
    report_content: str,
    report_id: str,
    reports_dir: str = "output/reports",
    prefix: str = "report"
) -> str:
    """
    Save report to file.

    Args:
        report_content: Report content (markdown, text, etc.)
        report_id: Unique identifier for the report
        reports_dir: Directory to save reports
        prefix: Filename prefix (default: "report")

    Returns:
        Path to saved report file
    """
    # Create reports directory if it doesn't exist
    reports_path = Path(reports_dir)
    reports_path.mkdir(parents=True, exist_ok=True)

    # Generate filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{prefix}_{report_id}_{timestamp}.md"
    filepath = reports_path / filename

    # Write report
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(report_content)

    return str(filepath)





# ✅ Report File Handling Utility — Summary

### **`save_report()`**

This utility’s entire job is to take a **finished report** (as text) and **save it to a file** in a clean, organized way.

It handles the boring-but-important stuff so your agent stays focused on logic, not file management.

---

# ⭐ What It Does

### **1. Ensures the reports folder exists**

If `"output/reports"` doesn’t exist, it creates it automatically.
→ No errors, no manual setup required.

---

### **2. Generates a smart, timestamped filename**

The filename looks like:

```
report_<report_id>_20250122_153022.md
```

This guarantees:

* Uniqueness
* Chronological ordering
* Easy retrieval
* Meaningful naming (prefix + id + timestamp)

---

### **3. Saves the report to disk**

It writes the report using UTF-8 encoding so all characters (even emojis, accents, symbols) are preserved.

Returns the **full file path** so the orchestrator can store it, display it, or forward it somewhere else.

---

# ⭐ Why It’s Useful

This utility gives your orchestrator:

* A consistent file naming scheme
* Automatic folder creation
* Reliable report storage
* Easy integration with dashboards, logs, emails, etc.

It's also completely reusable — perfect for your toolshed.

Any agent that produces output (reports, logs, summaries, audits) can use this exact same function.

---

# 🎯 In one sentence

**`save_report()` safely writes the orchestrator’s report to disk using clean folder structure and timestamped filenames.**


